In [1]:
import sys
# adding to the path variables the one folder higher (locally, not changing system variables)
sys.path.append("..")

import numpy as np
import pandas as pd
import datetime

import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.layers import Dense, Embedding, Dropout, LSTM, Input, Layer, BatchNormalization, Bidirectional, TextVectorization
import tensorflow.keras.backend as K
from tensorflow.keras.utils import plot_model

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
from mlxtend.plotting import plot_confusion_matrix
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
import seaborn as sns

print(tf.__version__)

# ignore the warnings
import warnings
warnings.filterwarnings("ignore")

# set Randomseed
RSEED = 42

# import needed functions
from scripts.processing import *

# Load the TensorBoard notebook extension
%load_ext tensorboard

2.8.0


In [2]:
# The maximum number of words to be used. (most frequent)
# TODO enlarge for larger datasets
VOCAB_SIZE = 100000

# Dimension of the dense embedding.
EMBEDDING_DIM = 128

# Max number of words in each review.
MAX_LENGTH = 200

# Define start and end index (end not included) of dataset
# DFR_START = None
DFR_END = 524288

# Define ANN hyperparameter
EPOCHS = 4
BATCH_SIZE = 512

In [3]:
# load the dataset and use specified dataset
# TODO change dfr to dataset when csv clean
dfr = pd.read_csv('../data/yelp_dataset/review_1819_preprocessed_final.csv')
# dfr = dfr[DFR_START:DFR_END]

In [4]:
# reduce dataset to text and rating
# TODO drop drop when csv clean
dataset = dfr.drop('Unnamed: 0', axis=1)
dataset = dataset.sample(DFR_END, random_state=RSEED)

# check dataset
dataset.head()

,text,useful
1493338,Awesome burgers the bun being an English muffi...,0
1122087,Fairly inexpensive asian food with a variety o...,1
724655,Stopped in for some sushi on a Wednesday night...,1
219475,Honestly Im very disappointed with this place....,1
879117,Okay I maybe the Grinch who stole Christmas bu...,1


In [5]:
# define feature and target
review = dataset[['text']]
# TODO delete apply after changes in csv
# target = dataset["useful"].apply(lambda x: 1 if x > 0 else 0).values
target = dataset["useful"]

In [6]:
# split the dataset into train and test set
X_train, X_test, y_train, y_test = train_test_split(review, target, test_size = 0.20, random_state = RSEED)
print(X_train.shape,y_train.shape)
print(X_test.shape,y_test.shape)

(419430, 1) (419430,)
(104858, 1) (104858,)


In [7]:
# initialize vectorizer
vectorize_layer = TextVectorization(
 standardize='lower_and_strip_punctuation', 
 ngrams=3,
 max_tokens=VOCAB_SIZE,
 output_mode='int',
 output_sequence_length=MAX_LENGTH)

# fit vectorizer
vectorize_layer.adapt(X_train)

Metal device set to: Apple M1

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB



2022-09-06 22:32:38.102247: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-09-06 22:32:38.102430: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2022-09-06 22:32:38.467990: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-09-06 22:32:38.562162: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


In [8]:
# SOURCE: https://machinelearningmastery.com/adding-a-custom-attention-layer-to-recurrent-neural-network-in-keras/
# Add attention layer to the deep learning network
class attention(Layer):
    def __init__(self,**kwargs):
        super(attention,self).__init__(**kwargs)

    def build(self,input_shape):
        self.W=self.add_weight(name='attention_weight', shape=(input_shape[-1],1), 
                               initializer='random_normal', trainable=True)
        self.b=self.add_weight(name='attention_bias', shape=(input_shape[1],1), 
                               initializer='zeros', trainable=True)        
        super(attention, self).build(input_shape)

    def call(self,x):
        # Alignment scores. Pass them through tanh function
        e = K.tanh(K.dot(x,self.W)+self.b)
        # Remove dimension of size 1
        e = K.squeeze(e, axis=-1)   
        # Compute the weights
        alpha = K.softmax(e)
        # Reshape to tensorFlow format
        alpha = K.expand_dims(alpha, axis=-1)
        # Compute the context vector
        context = x * alpha
        context = K.sum(context, axis=1)
        return context

In [9]:
# build model architecture
# TODO comment
text_in = Input(shape=(1,), dtype=tf.string)
vector = vectorize_layer(text_in)
embedding = Embedding(VOCAB_SIZE, EMBEDDING_DIM)(vector)
text_bidir_lstm_1 = Bidirectional(LSTM(128, return_sequences=True))(embedding)
text_dropout_1 = Dropout(0.5)(text_bidir_lstm_1)
text_bidir_lstm_2 = Bidirectional(LSTM(128, return_sequences=True))(text_dropout_1)
text_dropout_2 = Dropout(0.5)(text_bidir_lstm_2)
text_dense = Dense(64, activation='relu')(text_dropout_2)
text_batch_norm_1 = BatchNormalization()(text_dense)
text_out = Dropout(0.5)(text_batch_norm_1)

dense_1 = Dense(128, activation='relu')(text_out)
batch_norm_1 = BatchNormalization()(dense_1)
dropout_1 = Dropout(0.5)(batch_norm_1)
dense_2 = Dense(32, activation='relu')(dropout_1)
batch_norm_2 = BatchNormalization()(dense_2)
dropout_2 = Dropout(0.5)(batch_norm_2)

out = Dense(1, activation='sigmoid')(dropout_2)
model = Model(inputs=[text_in], outputs=[out])

In [10]:
#compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [11]:
# save history in tensorboard
# TODO get tensorboard working again
log_dir = "../logs/new/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

# Create a callback that saves the model's weights
# TODO save several checkpoints
checkpoint_path = "../training/model_extended_2_no_attention/cp.ckpt"
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

In [12]:
# set parameters and train the model
history = model.fit(X_train, y_train, shuffle=True ,
                    epochs=EPOCHS, batch_size=BATCH_SIZE, 
                    validation_split=0.2,
                    callbacks=[cp_callback])

model.save('../saved_model/model_extended_2_no_attention')

Epoch 1/4


2022-09-06 22:35:12.678819: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-09-06 22:35:13.611325: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-09-06 22:35:13.643603: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-09-06 22:35:14.725784: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-09-06 22:35:14.794855: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-09-06 22:35:20.541491: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-09-06 22:35:20.664342: I tensorflow/core/grappler/optimizers/cust

473/656 [====================>.........] - ETA: 11:41 - loss: 0.6836 - accuracy: 0.6121

KeyboardInterrupt: 

In [ ]:
# predict on test set
y_pred = (model.predict(X_test) > 0.5).astype('int32')

In [ ]:
# show the classification report
print(classification_report(y_test, y_pred))

# show confusion matrix
sns.heatmap(tf.math.confusion_matrix(y_test, y_pred), annot=True, fmt='g')

In [ ]:
!pip install pydot

In [ ]:
!pip install graphviz

In [ ]:
tf.keras.utils.plot_model(model, "multi_input_and_output_model_2.png", show_shapes=True)